# Exploring Hacker News Posts(Guided Project):
n this project, we'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).
Hacker News is a site started by the startup incubator [Hacker News](https://news.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

# Introduction
First, let's explore our dataset

Below are descriptions of the columns:

* __id__: The unique identifier from Hacker News for the post
* __title__: The title of the post
* __url__: The URL that the posts links to, if it the post has a URL
* __num_points__: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* __num_comments__: The number of comments that were made on the post
* __author__: The username of the person who submitted the post
* __created_at__: The date and time at which the post was submitted

In [1]:
# Read data
import csv
f = open('hacker_news.csv')
hn = list(csv.reader(f))
# Remove header
header = hn[0]
hn = hn[1:]
print(header)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We're specifically interested in posts whose titles begin with either <mark>Ask HN</mark> or <mark>Show HN</mark>. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

>Ask HN: How to improve my personal website?   
Ask HN: Am I the only one outraged by Twitter shutting down share counts?   
Ask HN: Aby recent changes to CSS that broke mobile?

Likewise, users submit <mark>Show HN</mark> posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

>Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'  
Show HN: Something pointless I made  
Show HN: Shanhu.io, a programming playground powered by e8vm  

We'll compare these two types of posts to determine the following:

* Do Ask HN or Show HN receive more comments on average?
* Do posts created at a certain time receive more comments on average?

# Extracting Ask HN and Show HN Posts
Now that we've removed the headers from <mark>hn</mark>, we're ready to filter our data. Since we're only concerned with post titles beginning with <mark>Ask HN</mark> or <mark>Show HN</mark>, we'll create new lists of lists containing just the data for those titles.

In [2]:
#splitting dataset

ask_posts=[]
show_posts=[]
other_posts=[]
#splitting criteria
for post in hn:
    title = post[1].lower()
    if title.startswith("ask hn"):
        ask_posts.append(post)
    elif title.startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
print("Number of \"ASK HN\" posts: "+str(len(ask_posts)))
print("Number of \"SHOW HN\" posts: "+str(len(show_posts)))
print("Number of other posts: "+str(len(other_posts)))

Number of "ASK HN" posts: 1744
Number of "SHOW HN" posts: 1162
Number of other posts: 17194


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [3]:
#average number of comments `Ask HN` posts receive.
total_ask_comments = 0 #total comments on ask posts
for post in ask_posts:
    total_ask_comments += int(post[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts) #average of ask comments
print("Average number of comments on \"ASK HN\" posts: "+
      str(round(avg_ask_comments,2))+" comments")

Average number of comments on "ASK HN" posts: 14.04 comments


In [4]:
#average number of comments `Show HN` posts receive.
total_show_comments = 0 #total comments on show posts
for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts) #average of show comments
print("Average comments on \"SHOW HN\" posts: "+
      str(round(avg_show_comments,2))+" comments")

Average comments on "SHOW HN" posts: 10.32 comments


On average, ask posts in our sample received approximately 14 comments per post, on the other hand show posts received approximately 10 comments. <mark>ASK HN</mark> posts are more lik.

# Finding the Amount of Ask Posts and Comments by Hour Created

Since <mark>ASK HN</mark> has more reach, so we will focus on it for the rest of the analysis.  
Next, we'll determine if ask posts created at a certain time are more likely to attract comments.

In [5]:
#parse date of creation and number of comments
import datetime as dt
result_list=[]
for post in ask_posts:
    created_at=post[6]
    comments_n=int(post[4])
    result_list.append([created_at,comments_n])
counts_by_hour={} #count of posts per hour
comments_by_hour={} #count of comments per hour
date_format = "%m/%d/%Y %H:%M"
for result in result_list:
    date=result[0]
    date=dt.datetime.strptime(date,date_format)
    hour=date.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour]=1
        comments_by_hour[hour]=result[1] #adding number of comments of post
    else:
        counts_by_hour[hour]+=1
        comments_by_hour[hour]+=result[1] #adding number of comments of post
comments_by_hour

{'00': 447,
 '01': 683,
 '02': 1381,
 '03': 421,
 '04': 337,
 '05': 464,
 '06': 397,
 '07': 267,
 '08': 492,
 '09': 251,
 '10': 793,
 '11': 641,
 '12': 687,
 '13': 1253,
 '14': 1416,
 '15': 4477,
 '16': 1814,
 '17': 1146,
 '18': 1439,
 '19': 1188,
 '20': 1722,
 '21': 1745,
 '22': 479,
 '23': 543}

# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [6]:
avg_by_hour=[]
for hour in comments_by_hour:
    avg_by_hour.append(
        [hour,
         round(comments_by_hour[hour]/counts_by_hour[hour],2)])#calc avg comments per hour
sorted(avg_by_hour)

[['00', 8.13],
 ['01', 11.38],
 ['02', 23.81],
 ['03', 7.8],
 ['04', 7.17],
 ['05', 10.09],
 ['06', 9.02],
 ['07', 7.85],
 ['08', 10.25],
 ['09', 5.58],
 ['10', 13.44],
 ['11', 11.05],
 ['12', 9.41],
 ['13', 14.74],
 ['14', 13.23],
 ['15', 38.59],
 ['16', 16.8],
 ['17', 11.46],
 ['18', 13.2],
 ['19', 10.8],
 ['20', 21.52],
 ['21', 16.01],
 ['22', 6.75],
 ['23', 7.99]]

# Sorting and Printing Values from a List of Lists

In [7]:
swap_avg_by_hour=[]
for hour in avg_by_hour:
    swap_avg_by_hour.append([hour[1], hour[0]])
print(swap_avg_by_hour)
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

[[14.74, '13'], [11.05, '11'], [11.46, '17'], [7.17, '04'], [9.41, '12'], [38.59, '15'], [13.23, '14'], [7.99, '23'], [13.44, '10'], [7.85, '07'], [16.01, '21'], [6.75, '22'], [23.81, '02'], [13.2, '18'], [5.58, '09'], [10.09, '05'], [9.02, '06'], [21.52, '20'], [8.13, '00'], [10.8, '19'], [11.38, '01'], [10.25, '08'], [16.8, '16'], [7.8, '03']]


In [10]:
print("Top 5 Hours for 'Ask HN' Comments")
template="{time}: {avg:.2f} average comments per post"
for avg, hr in sorted_swap[:5]:
    print(template.format(
            time=dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg=avg))

Top 5 Hours for 'Ask HN' Comments
15:00:38.59 average comments per post
02:00:23.81 average comments per post
20:00:21.52 average comments per post
16:00:16.80 average comments per post
21:00:16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post.  
There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.
According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US.

# Conclusion
In this project, we analyzed <mark>ask posts</mark> and <mark>show posts</mark> to determine which type of posts and time received the most comments on average.  
Based on our analysis, the maximum amount of comments a post receives, happen when the post is categorized as <mark>ask post</mark> and created between 15:00 and 16:00 est.

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, <mark>ask posts</mark> received more comments on average, on top of that <mark>ask posts</mark> between 15:00 and 16:00 est. received the most comments on average.